In [1]:
!pip install streamlit pyngrok joblib pandas scikit-learn lightgbm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import pickle

df = pd.read_csv("final dataset.csv")

district_state_map = (
    df[["District", "State_x"]]
    .drop_duplicates()
    .set_index("District")["State_x"]
    .to_dict()
)

pickle.dump(district_state_map, open("district_state_map.pkl", "wb"))

print(f"Saved mapping for {len(district_state_map)} districts.")




Saved mapping for 88 districts.


In [4]:
%%writefile app.py
import streamlit as st
import joblib
import os
import pandas as pd
import pickle

# -----------------------------
# Load district→state mapping
# -----------------------------
district_state_map = pickle.load(open("district_state_map.pkl", "rb"))

# -----------------------------
# 2022 Environmental Values
# -----------------------------
ENV_2022 = {
    "Uttar Pradesh":  {"NDVI": 4798.719532, "Rainfall": 877.0945416, "Temperature": 28.66469342},
    "Punjab":         {"NDVI": 5846.595107, "Rainfall": 710.1282511, "Temperature": 29.40444777},
    "Haryana":        {"NDVI": 4310.983064, "Rainfall": 565.8587185, "Temperature": 30.1846854}
}

MODEL_DIR = "crop_models"

# -----------------------------
# Load all crop models
# -----------------------------
@st.cache_resource
def load_all_models():
    models = {}
    for f in os.listdir(MODEL_DIR):
        if f.endswith(".pkl"):
            crop = f[:-4]
            models[crop] = joblib.load(os.path.join(MODEL_DIR, f))
    return models

models = load_all_models()

# -----------------------------
# UI
# -----------------------------
st.title("🌾 Crop Recommendation (State-based, 2022 Conditions)")
st.write("Enter State & District to get best crops for your region.")

state = st.selectbox("Select State", ["Uttar Pradesh", "Punjab", "Haryana"])
district = st.text_input("Enter District Name (case-sensitive)")

if st.button("Recommend Crops"):
    if district == "":
        st.error("❌ Please enter a district.")
    else:
        # Get ENV for selected state
        env = ENV_2022[state]

        # Pipeline requires this exact structure
        input_row = pd.DataFrame([{
            "State_x": state,
            "District": district,
            "NDVI": env["NDVI"],
            "Rainfall": env["Rainfall"],
            "Temperature": env["Temperature"],
            "Area": 1
        }])

        preds = []

        # Run prediction for all crop models
        for crop, model in models.items():
            try:
                pred = model.predict(input_row)[0]
                preds.append((crop, pred))
            except:
                pass

        if len(preds) == 0:
            st.error("⚠ No predictions generated. District may be incorrect.")
        else:
            preds = sorted(preds, key=lambda x: x[1], reverse=True)

            st.subheader("🌱 Top 3 Crops")
            for crop, val in preds[:3]:
                st.write(f"**{crop}** → {val:.2f}")

            st.subheader("📊 All Predictions")
            st.dataframe(pd.DataFrame(preds, columns=["Crop", "Predicted Yield"]))


Writing app.py


In [5]:
from pyngrok import ngrok

NGROK_TOKEN = "35SnqVIxpGlKfh9k3k9T7GOMh01_6D52rcVWyaGv7Pyh5Ck1b"
ngrok.set_auth_token(NGROK_TOKEN)

!mkdir -p /content/logs

# Run Streamlit app in background
!streamlit run app.py &>/content/logs/log.txt &

# Connect ngrok to port 8501
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://unverbalized-geoffrey-subspirally.ngrok-free.dev" -> "http://localhost:8501">